In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score, roc_auc_score

# Wczytanie danych
url = "https://raw.githubusercontent.com/lorenzodenisi/Heart-Failure-Clinical-Records/refs/heads/master/heart_failure_clinical_records_dataset.csv"
df = pd.read_csv(url)
print(df.head())

# Podział na cechy (X) i etykiety (y)
X = df.drop('DEATH_EVENT', axis=1)
y = df['DEATH_EVENT']

# Podział na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)





    age  anaemia  creatinine_phosphokinase  diabetes  ejection_fraction  \
0  75.0        0                       582         0                 20   
1  55.0        0                      7861         0                 38   
2  65.0        0                       146         0                 20   
3  50.0        1                       111         0                 20   
4  65.0        1                       160         1                 20   

   high_blood_pressure  platelets  serum_creatinine  serum_sodium  sex  \
0                    1  265000.00               1.9           130    1   
1                    0  263358.03               1.1           136    1   
2                    0  162000.00               1.3           129    1   
3                    0  210000.00               1.9           137    1   
4                    0  327000.00               2.7           116    0   

   smoking  time  DEATH_EVENT  
0        0     4            1  
1        0     6            1  
2       

 zad 1 kluczowa jest relacja między poszczególnymi wartościami cech, a nie ich bezwzględna skala. Drzewo decyzyjne dokonuje podziałów danych na podstawie warunków typu „czy wartość cechy jest większa/mniejsza od pewnej liczby”, co oznacza, że kolejność oraz relacja między wartościami są zachowane, niezależnie od tego, czy dane są standaryzowane, czy nie. W przeciwieństwie do algorytmów takich jak SVM, które wykorzystują odległości między punktami w przestrzeni

In [5]:
def evaluate_model(model, X_test, y_test, model_name):
    # Predykcje
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]  # Prawdopodobieństwo dla klasy 1

    # Obliczenia metryk
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_prob)

    # Wypisanie wyników
    print(f"{model_name} Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, ROC AUC: {roc_auc:.4f}")
    print(f"{model_name} Classification Report:\n", classification_report(y_test, y_pred))


# AdaBoost
adaboost = AdaBoostClassifier(random_state=42)
adaboost.fit(X_train, y_train)
evaluate_model(adaboost, X_test, y_test, "AdaBoost")

# Gradient Boosting
gradient_boosting = GradientBoostingClassifier(random_state=42)
gradient_boosting.fit(X_train, y_train)
evaluate_model(gradient_boosting, X_test, y_test, "Gradient Boosting")

# XGBoost
xgboost = XGBClassifier(random_state=42)
xgboost.fit(X_train, y_train)
evaluate_model(xgboost, X_test, y_test, "XGBoost")

# LightGBM
lightgbm = LGBMClassifier(random_state=42, verbose=-1)
lightgbm.fit(X_train, y_train)
evaluate_model(lightgbm, X_test, y_test, "LightGBM")


AdaBoost Accuracy: 0.7667, F1-score: 0.6667, ROC AUC: 0.8640
AdaBoost Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.91      0.82        35
           1       0.82      0.56      0.67        25

    accuracy                           0.77        60
   macro avg       0.78      0.74      0.74        60
weighted avg       0.78      0.77      0.76        60

Gradient Boosting Accuracy: 0.7167, F1-score: 0.5854, ROC AUC: 0.8823
Gradient Boosting Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.89      0.78        35
           1       0.75      0.48      0.59        25

    accuracy                           0.72        60
   macro avg       0.73      0.68      0.69        60
weighted avg       0.72      0.72      0.70        60

XGBoost Accuracy: 0.7667, F1-score: 0.6818, ROC AUC: 0.8823
XGBoost Classification Report:
               precision    recall  f1-score   su

zad 2 Najlepsze wyniki: XGBoost i LightGBM, które uzyskały wyższą precyzję (względnie w f1-score) i nieco lepszy ROC AUC.

Różnice między metodami: Różnice są raczej niewielkie – wszystkie metody (poza Gradient Boosting) uzyskały porównywalne wyniki pod względem accuracy. Jednakże, patrząc na f1-score oraz ROC AUC, LightGBM (i XGBoost) mają przewagę, choć poprawa nie jest drastyczna.

zad 3 Choć różnice nie są ogromne, modyfikacja hiperparametrów (zwiększenie liczby estymatorów do 300 oraz ustawienie learning_rate na 0.2) przyniosła zauważalną poprawę rezultatów klasyfikacji

In [6]:
modified_adaboost = AdaBoostClassifier(random_state=42, n_estimators=300, learning_rate=0.2)
modified_adaboost.fit(X_train, y_train)
print("\nModified AdaBoost:")
evaluate_model(modified_adaboost, X_test, y_test, "Modified AdaBoost")


Modified AdaBoost:
Modified AdaBoost Accuracy: 0.8000, F1-score: 0.7000, ROC AUC: 0.8674
Modified AdaBoost Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.97      0.85        35
           1       0.93      0.56      0.70        25

    accuracy                           0.80        60
   macro avg       0.84      0.77      0.77        60
weighted avg       0.83      0.80      0.79        60



zad 4 ✓ Zadanie 4. Zmodyfikuj hiperparamtery innych modeli. Spróbuj dopasować wartości hiperparametrów aby zmaksymalizować wartości współczynników jakości modelu. Jakie wyniki udało Ci się uzyskać? Dla której metody modelowania drzew wzmacnianych? Porównaj wyniki z innymi osobami w sali.

In [8]:
# Tuned Gradient Boosting
tuned_gradient = GradientBoostingClassifier(
    random_state=42,
    n_estimators=200,
    learning_rate=0.1,
    max_depth=3
)
tuned_gradient.fit(X_train, y_train)
evaluate_model(tuned_gradient, X_test, y_test, "Tuned Gradient Boosting")

# Tuned XGBoost
tuned_xgb = XGBClassifier(
    random_state=42,
    n_estimators=300,
    learning_rate=0.1,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric='logloss'
)
tuned_xgb.fit(X_train, y_train)
evaluate_model(tuned_xgb, X_test, y_test, "Tuned XGBoost")

# Tuned LightGBM
tuned_lgbm = LGBMClassifier(
    random_state=42,
    n_estimators=300,
    learning_rate=0.1,
    num_leaves=31
)
tuned_lgbm.fit(X_train, y_train)
evaluate_model(tuned_lgbm, X_test, y_test, "Tuned LightGBM")


Tuned Gradient Boosting Accuracy: 0.7333, F1-score: 0.6190, ROC AUC: 0.8811
Tuned Gradient Boosting Classification Report:
               precision    recall  f1-score   support

           0       0.72      0.89      0.79        35
           1       0.76      0.52      0.62        25

    accuracy                           0.73        60
   macro avg       0.74      0.70      0.71        60
weighted avg       0.74      0.73      0.72        60



/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [06:41:10] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Tuned XGBoost Accuracy: 0.8000, F1-score: 0.7273, ROC AUC: 0.8880
Tuned XGBoost Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.91      0.84        35
           1       0.84      0.64      0.73        25

    accuracy                           0.80        60
   macro avg       0.81      0.78      0.78        60
weighted avg       0.81      0.80      0.79        60

Tuned LightGBM Accuracy: 0.7667, F1-score: 0.6818, ROC AUC: 0.8800
Tuned LightGBM Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.89      0.82        35
           1       0.79      0.60      0.68        25

    accuracy                           0.77        60
   macro avg       0.77      0.74      0.75        60
weighted avg       0.77      0.77      0.76        60



Tuned Gradient Boosting osiąga Accuracy: 0.7333, F1-score: 0.6190 oraz ROC AUC: 0.8811.

Tuned XGBoost uzyskuje Accuracy: 0.8000, F1-score: 0.7273 oraz ROC AUC: 0.8880.

Tuned LightGBM daje Accuracy: 0.7667, F1-score: 0.6818 oraz ROC AUC: 0.8800.

Widać, że najlepsze rezultaty uzyskano dla Tuned XGBoost, który wyróżnia się najwyższym F1-score i

zad 5 Zadanie 5. Podobne eksperymenty wykonaj dla pliku z większą liczbą klas np.:https://raw. githubusercontent.com/Ayantika22/PCA-Principle-Component-Analysis-For-Wine-dataset/ refs/heads/master/Wine.csv. Aby uporać się z problemem wieloklasowym należy przerobić kod funkcji evaluete model. Poza tym dla klasyfikatora XGBoost należy przerobić zmienną odpowiedzi jeśli etykiety nie zaczynają się od 0. Można porzucić ten model lub spróbować dopisać kod modyfikujący etykietowanie danych w problemie wieloklasowym.

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, f1_score, roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Wczytanie danych Wine
url_wine = "https://raw.githubusercontent.com/Ayantika22/PCA-Principle-Component-Analysis-For-Wine-dataset/refs/heads/master/Wine.csv"
wine_df = pd.read_csv(url_wine)

# Podział na cechy i etykiety
X_wine = wine_df.drop('Customer_Segment', axis=1)
y_wine = wine_df['Customer_Segment']

# Mapowanie etykiet, jeśli nie zaczynają się od 0 (konieczne dla XGBoost)
if y_wine.min() != 0:
    mapping = {label: idx for idx, label in enumerate(sorted(y_wine.unique()))}
    y_wine = y_wine.map(mapping)

# Podział danych na zbiór treningowy i testowy
X_train_wine, X_test_wine, y_train_wine, y_test_wine = train_test_split(X_wine, y_wine, test_size=0.2, random_state=42)

def evaluate_model_multiclass(model, X_test, y_test, model_name):
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    # Obliczenie ROC AUC dla problemu wieloklasowego
    try:
        roc_auc = roc_auc_score(y_test, y_prob, multi_class='ovr')
    except Exception as e:
        roc_auc = None
        print("ROC AUC could not be computed:", e)

    print(f"{model_name} Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}", end="")
    if roc_auc is not None:
        print(f", ROC AUC: {roc_auc:.4f}")
    else:
        print("")
    print(f"{model_name} Classification Report:\n", classification_report(y_test, y_pred))

# Przykładowy model Gradient Boosting
gb_model = GradientBoostingClassifier(random_state=42, n_estimators=150, learning_rate=0.1, max_depth=3)
gb_model.fit(X_train_wine, y_train_wine)
evaluate_model_multiclass(gb_model, X_test_wine, y_test_wine, "Gradient Boosting (Wine)")

# Przykładowy model XGBoost (po modyfikacji etykiet)
xgb_model = XGBClassifier(
    random_state=42,
    n_estimators=150,
    learning_rate=0.1,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric='mlogloss'
)
xgb_model.fit(X_train_wine, y_train_wine)
evaluate_model_multiclass(xgb_model, X_test_wine, y_test_wine, "XGBoost (Wine)")


Gradient Boosting (Wine) Accuracy: 0.9444, F1-score: 0.9440, ROC AUC: 0.9913
Gradient Boosting (Wine) Classification Report:
               precision    recall  f1-score   support

           0       0.93      1.00      0.97        14
           1       0.93      0.93      0.93        14
           2       1.00      0.88      0.93         8

    accuracy                           0.94        36
   macro avg       0.95      0.93      0.94        36
weighted avg       0.95      0.94      0.94        36



/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [06:44:12] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost (Wine) Accuracy: 1.0000, F1-score: 1.0000, ROC AUC: 1.0000
XGBoost (Wine) Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       1.00      1.00      1.00        14
           2       1.00      1.00      1.00         8

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.00        36
weighted avg       1.00      1.00      1.00        36



Model XGBoost osiąga idealne wyniki na zbiorze testowym, co może oznaczać, że model doskonale dopasował się do danych testowych. Jednakże takie wyniki należy interpretować ostrożnie – istnieje ryzyko przeuczenia (overfittingu), szczególnie jeśli zbiór testowy jest niewielki (w tym przypadku 36 próbek).

Podsumowując, mimo że XGBoost uzyskał perfekcyjne wyniki, warto zwrócić uwagę na wielkość zbioru testowego oraz ewentualną potrzebę walidacji krzyżowej, aby upewnić się, że rezultaty są stabilne.